## 5. 뉴스 카테고리 다중분류-Project

In [1]:
from tensorflow.keras.datasets import reuters
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB #다항분포 나이브 베이즈 모델
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score #정확도 계산


from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

2023-06-28 17:06:25.341621: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
word_index = reuters.get_word_index(path="reuters_word_index.json")
index_to_word = { index+3 : word for word, index in word_index.items() }

for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
    index_to_word[index]=token

In [7]:
dtmvector = CountVectorizer()
tfidf_transformer = TfidfTransformer()

def reuters_load_ml(num_words):
    (x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=num_words, test_split=0.2)
    
    decoded = []
    for i in range(len(x_train)):
        t = ' '.join([index_to_word[i] for index in x_train[i]])
        decoded.append(t)

    x_train = decoded
    
    decoded = []
    for i in range(len(x_test)):
        t = ' '.join([index_to_word[i] for index in x_test[i]])
        decoded.append(t)

    x_test = decoded
    
    x_train_dtm = dtmvector.fit_transform(x_train)
    x_train = tfidf_transformer.fit_transform(x_train_dtm)
    x_test_dtm = dtmvector.transform(x_test) #테스트 데이터를 DTM으로 변환
    x_test = tfidf_transformer.transform(x_test_dtm) #DTM을 TF-IDF 행렬로 변환
    
    return x_train, y_train, x_test, y_test


In [8]:
def fit_ml(x_train, y_train, x_test, y_test):
    nb = MultinomialNB()
    nb.fit(x_train, y_train)
    predicted = nb.predict(x_test) #테스트 데이터에 대한 예측
    
    cb = ComplementNB()
    cb.fit(x_train, y_train)
    predicted = cb.predict(x_test)
    
    lr = LogisticRegression(C=10000, penalty='l2', max_iter=3000)
    lr.fit(x_train, y_train)
    predicted = lr.predict(x_test)
    
    lsvc = LinearSVC(C=1000, penalty='l1', max_iter=3000, dual=False)
    lsvc.fit(x_train, y_train)
    predicted = lsvc.predict(x_test)
    
    tree = DecisionTreeClassifier(max_depth=10, random_state=0)
    tree.fit(x_train, y_train)
    predicted = tree.predict(x_test)
    
    forest = RandomForestClassifier(n_estimators=5, random_state=0)
    forest.fit(x_train, y_train)
    predicted = forest.predict(x_test)
    
    grbt = GradientBoostingClassifier(random_state=0) # verbose=3
    grbt.fit(x_train, y_train)
    predicted = grbt.predict(x_test)
    
    clf1 = LogisticRegression()
    clf2 = ComplementNB()
    clf3 = GradientBoostingClassifier(random_state=0)

    voting_classifier = VotingClassifier(estimators=[('lr', clf1), ('gnb', clf2), ('dt', clf3)], voting='hard')
    voting_classifier.fit(x_train, y_train)
    predicted = voting_classifier.predict(x_test)

    

In [9]:
x_train, y_train, x_test, y_test = reuters_load_ml(None)
#fit_ml(x_train, y_train, x_test, y_test)

KeyError: 3

In [ ]:
x_train, y_train, x_test, y_test = reuters_load_ml(None)
fit_ml(x_train, y_train, x_test, y_test)

x_train, y_train, x_test, y_test = reuters_load_ml(None)
fit_ml(x_train, y_train, x_test, y_test)

x_train, y_train, x_test, y_test = reuters_load_ml(10000)
fit_ml(x_train, y_train, x_test, y_test)

x_train, y_train, x_test, y_test = reuters_load_ml(10000)
fit_ml(x_train, y_train, x_test, y_test)


x_train, y_train, x_test, y_test = reuters_load_ml(50000)
fit_ml(x_train, y_train, x_test, y_test)


x_train, y_train, x_test, y_test = reuters_load_ml(5000)
fit_ml(x_train, y_train, x_test, y_test)


## F1-Score, Confusion Matrix

### 필요한 라이브러리 불러오기

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
print(classification_report(y_test, model.predict(tfidfv_test), zero_division=0))

In [ ]:
def graph_confusion_matrix(model, x_test, y_test):#, classes_name):
  df_cm = pd.DataFrame(confusion_matrix(y_test, model.predict(x_test)))#, index=classes_name, columns=classes_name)
  fig = plt.figure(figsize=(12,12))
  heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
  heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=12)
  heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=12)
  plt.ylabel('label')
  plt.xlabel('predicted value')

In [ ]:
graph_confusion_matrix(model, tfidfv_test, y_test)

In [ ]:
predicted = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교